In [22]:
!pip install openai==0.28

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Whisper, STT API를 활용한 음성 인식 및 오류 교정

##### 1. OPEN AI Whisper api 사용해보기

In [24]:
OPENAI_API_KEY='sk-'

In [7]:
import openai

audio_path = "/content/drive/MyDrive/데이터/ch0601/CH03-07_sound_cut.mp3"

with open(audio_path, "rb") as audio_file:
    transcript = openai.Audio.transcribe(
        file = audio_file,
        model = "whisper-1",
        response_format="text",
        language="ko",
        api_key = OPENAI_API_KEY
    )
print(transcript)



3번째 챕터의 7번째 클립, RAG, 자체 문서를 위한 커스텀 검색 시스템 제작 강의 시작하겠습니다. 먼저 개념에 대해서 설명드리고 실습으로 넘어가도록 하겠습니다. 외부 정보 통합 자동화의 필요성에 대해서 먼저 설명을 드리고 시작을 해보도록 하겠습니다. 그래서 지금까지의 강의에서 계속해서 다뤘던 내용들은 왼쪽에서 보시다시피 사용자가 어떤 특정 질문을 던지게 되고 상담복이 회사에서 가지고 있는 데이터베이스로부터 정보를 받아서 응답을 하는 이런 시나리오를 많이 보여드렸습니다. 근데 생각을 해보시면, 저희가 실습했던 내용을 생각해보시면 저희가 직접 어떤 문서를 봐야 해당 질문에 대해서 답변을 할 수 있는지 직접 그 문서를 골라서 프롬프트에 투입을 해줬던 것으로 기억을 하실 겁니다. 저번 시간에 있던 RAG를 제외하고서는요. 그래서 그 질문에 대해서 답변할 수 있는 문서들은 회사에서 가지고 있는 매뉴얼이라던가 뭐 다양한 정보들이 있을 건데 그 정보들이 상당히 다양하기 때문에 항상 상담복에게 프롬프트에 인풋으로 넣어줄 수는 없을 거고요. 그렇기 때문에 저희는 질문에 따라서 필요한 문서가 어떤 것인지를 검색해서 그 검색된 결과에 따라서 답변을 생성할 수 있는 이런 외부정보 통합 과정을 자동화할 필요성은 상당히 중요하다라고 말씀을 드릴 수 있겠습니다. 그래서 이 과정을 어떻게 할 수 있을까요? 이 데이터베이스와 상담복과의 인터랙션, 상호작용을 어떻게 디자인을 할 수 있을지에 대해서 오늘 강의에서 설명을 드려보도록 하겠습니다. 그래서 검색 시스템에 대해서 알아야 되겠죠? 검색 시스템에 대해서 크게 설명을 드리기 위해서 두 가지 분류를 먼저 나눠서 설명을 드려보겠습니다. 그래서 스팔스 리트리버와 댄스 리트리버, 그러니까 저번 시간에도 설명드렸다시피 검색이라는 단어를 자연어 처리에서는 Retriever, 반환이라는 단어를 쓴다고 말씀을 드렸죠. 그래서 스팔스랑 댄스라는 것을 한국말로 번역을 하기에는 좀 어려워서 그대로 가지고 왔습니다. 그래서 설명을 드려보면요. 첫 번째로 스팔

##### 2. 문법 교정 하기


In [26]:
keywords = ["상담복: 상담봇", "스팔스 리트리버: Sparse Retriever", "덴스 리트리버: Dense Retriever", "프리퀀시: Frequency"]

prompt = ""
for keyword in keywords:
  prompt += keyword + ", "

print(prompt)

상담복: 상담봇, 스팔스 리트리버: Sparse Retriever, 덴스 리트리버: Dense Retriever, 프리퀀시: Frequency, 


In [ ]:
with open(audio_path, "rb") as audio_file:
    transcript = openai.Audio.transcribe(
        prompt=prompt,
        file = audio_file,
        model = "whisper-1",
        response_format="text",
        language="ko",
        api_key = OPENAI_API_KEY
    )
print(transcript)


3번째 챕터의 7번째 클립, RAG, 자체 문서를 위한 커스텀 검색 시스템 제작 강의 시작하겠습니다. 먼저 개념에 대해서 설명드리고 실습으로 넘어가도록 하겠습니다. 외부 정보 통합 자동화의 필요성에 대해서 먼저 설명을 드리고 시작을 해보도록 하겠습니다. 그래서 지금까지의 강의에서 계속해서 다뤘던 내용들은 왼쪽에서 보시다시피 사용자가 어떤 특정 질문을 던지게 되고 상담봇이 회사에서 가지고 있는 데이터베이스로부터 정보를 받아서 응답을 하는 이런 시나리오를 많이 보여드렸습니다. 그런데 생각을 해보시면 저희가 실습했던 내용을 생각해보시면 저희가 직접 어떤 문서를 봐야 해당 질문에 대해서 답변을 할 수 있는지 직접 그 문서를 골라서 프롬프트에 투입을 해줬던 것으로 기억하실 겁니다. 저번 시간에 있던 RAG를 제외하고서는요. 그래서 그 질문에 대해서 답변할 수 있는 문서들은 회사에서 가지고 있는 매뉴얼이라던가 다양한 정보들이 있을 건데 그 정보들이 상당히 다양하기 때문에 항상 상담봇에게 프롬프트에 인풋으로 넣어줄 수는 없을 거고요. 그렇기 때문에 저희는 질문에 따라서 필요한 문서가 어떤 것인지를 검색해서 그 검색된 결과에 따라서 답변을 생성할 수 있는 외부 정보 통합 과정을 자동화할 필요성은 상당히 중요하다라고 말씀드릴 수 있겠습니다. 그래서 이 과정을 어떻게 할 수 있을까요? 데이터베이스와 상담봇과의 인터랙션, 상호작용을 어떻게 디자인을 할 수 있을지에 대해서 오늘 강의에서 설명을 드려보도록 하겠습니다. 그래서 검색 시스템에 대해서 알아야 되겠죠? 검색 시스템에 대해서 크게 설명을 드리기 위해서 두 가지 분류를 먼저 나눠서 설명을 드려보겠습니다. 그래서 Sparse Retrieval과 Dense Retrieval, 그러니까 저번 시간에도 설명드렸다시피 검색이라는 단어를 자연어 처리에서는 Retrieval, 반환이라는 단어를 쓴다고 말씀을 드렸죠? 그래서 Sparse랑 Dense라는 것을 한국말로 번역을 하기에는 어려워서 그대로 가지고 왔습니다. 그래서 설명을 드려보면요

### 영어에서 가능한 설정들
- 스타일 변화 주기: in casual style
- separator 설정: Instead of periods, end every sentence with elipses.


##### 3. 언어 모델을 이용하여 STT 결과 요약하기

In [27]:
import openai
chat_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "안녕하세요"}],
    api_key = OPENAI_API_KEY
)

print(chat_completion["choices"][0]["message"]["content"])

안녕하세요! 무엇을 도와드릴까요? :)


In [28]:
from string import Template

summary_prompt = Template("""
아래 내용은 사용자의 음성을 텍스트로 변환한 내용입니다.

내용을 요약해 주세요.

내용: $input
요약:
""")

print(summary_prompt.substitute(input="{투입될 텍스트가 입력될 자리입니다}"))


아래 내용은 사용자의 음성을 텍스트로 변환한 내용입니다.

내용을 요약해 주세요.

내용: {투입될 텍스트가 입력될 자리입니다}
요약:



In [29]:
def summary_voice(voice_path):
  with open(audio_path, "rb") as audio_file:
    transcript = openai.Audio.transcribe(
        prompt=prompt,
        file = audio_file,
        model = "whisper-1",
        response_format="text",
        language="ko",
        api_key = OPENAI_API_KEY
    )

  chat_completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": summary_prompt.substitute(input=transcript)}],
      api_key = OPENAI_API_KEY
  )

  return chat_completion["choices"][0]["message"]["content"]

print(summary_voice("/content/drive/MyDrive/데이터/ch0601/CH03-07_sound_cut.mp3"))

3번째 챕터의 7번째 클립에서는 RAG 자체 문서를 위한 커스텀 검색 시스템 제작 강의가 시작됩니다. 외부 정보 통합 자동화의 필요성을 강조하며, 스팔스 리트리버와 댄스 리트리버를 통해 검색 시스템의 두 가지 분류를 설명합니다. 스팔스 리트리버는 직접적인 키워드 일치를 통해 검색하고, 댄스 리트리버는 임베딩 벡터를 통해 의미적 유사성을 고려한 검색을 합니다. 임베딩은 데이터를 저차원 수치적 벡터로 표현하는 과정이며, 컴퓨터가 이해할 수 있는 값으로 표현하는 과정이라고 설명됩니다.


**Reference**

https://cookbook.openai.com/examples/whisper_prompting_guide